In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [2]:
df = pd.read_parquet('/Users/harshitgupta/Desktop/vs /VS-Data-Den/dates_grouped_data.parquet')

In [23]:
df.columns

Index(['PTNR_RM_CODE', 'WEEK', 'SALES', 'FLAG'], dtype='object')

In [3]:
df = df.set_index('DATE_FIELD')

In [4]:
df.head()

,PTNR_RM_CODE,WEEK,SALES,FLAG
DATE_FIELD,,,,
2021-01-31,0.0,1,1497279.48,Active
2021-02-01,0.0,1,1352352.29,Active
2021-02-02,0.0,1,1336891.59,Active
2021-02-03,0.0,1,1333770.96,Active
2021-02-04,0.0,1,1468008.14,Active


In [5]:
df['FLAG'] = df['FLAG'].map({'Inactive': 0, 'Active': 1})

In [6]:
for i in range(len(df.columns)):
  result = adfuller(df[df.columns[i]])

  if result[1] > 0.05 :
    print('{} - Series is not Stationary'.format(df.columns[i]))
  else:
    print('{} - Series is Stationary'.format(df.columns[i]))

PTNR_RM_CODE - Series is Stationary
WEEK - Series is Stationary
SALES - Series is Stationary
FLAG - Series is Stationary


In [82]:
max_lags=12
y='SALES'

In [83]:
df_input=df[['WEEK','PTNR_RM_CODE','SALES','FLAG']]

In [84]:
df_input


,WEEK,PTNR_RM_CODE,SALES,FLAG
DATE_FIELD,,,,
2021-01-31,1,0.0,1497279.48,1
2021-02-01,1,0.0,1352352.29,1
2021-02-02,1,0.0,1336891.59,1
2021-02-03,1,0.0,1333770.96,1
2021-02-04,1,0.0,1468008.14,1
...,...,...,...,...
2023-01-16,51,50001145.0,4100.67,1
2023-01-17,51,50001145.0,3619.68,1
2023-01-18,51,50001145.0,3607.62,1


In [85]:
df_train = df_input[:int(0.9*(len(df_input)))]
df_test = df_input[int(0.9*(len(df_input))):]

In [86]:
df_train.shape

(7711, 4)

In [87]:
from sklearn.metrics import mean_absolute_percentage_error

In [88]:
cols = df_train.columns

In [89]:
best_aic = float('inf')
best_bic = float('inf')
best_order = 0
best_model = None
best_mape = float('inf')

In [90]:
for i in range(1, max_lags + 1):
    print('Order =', i)
    model = VAR(df_train)
    results = model.fit(i)
    print('AIC:', results.aic)
    print('BIC:', results.bic)
    

    lag_order = results.k_ar
    forecast = results.forecast(df_train.values[-lag_order:], len(df_test))
    
 
    mape_val = mean_absolute_percentage_error(df_test[cols].values, forecast)
    print('MAPE:', mape_val)
    print('-' * 40)
    
    # Update best values if necessary
    if results.aic < best_aic:
        best_aic = results.aic
    if results.bic < best_bic:
        best_bic = results.bic
    if mape_val < best_mape:
        best_mape = mape_val
        best_order = i
        best_model = results

Order = 1
AIC: 49.59829099391477
BIC: 49.61632023766719
MAPE: 3.753278985563126e+18
----------------------------------------
Order = 2
AIC: 49.480377302060845
BIC: 49.51283354479434
MAPE: 3.7627945292257157e+18
----------------------------------------
Order = 3
AIC: 49.43616379165945
BIC: 49.48305023814827
MAPE: 3.765796263536948e+18
----------------------------------------
Order = 4
AIC: 49.42650521083834
BIC: 49.48782506699935
MAPE: 3.7654885219260846e+18
----------------------------------------
Order = 5
AIC: 49.423997211050654
BIC: 49.499753683943936
MAPE: 3.7689678679537367e+18
----------------------------------------
Order = 6
AIC: 49.42550682158889
BIC: 49.515703119418276
MAPE: 3.76973722966276e+18
----------------------------------------
Order = 7
AIC: 49.41663727119374
BIC: 49.52127660330738
MAPE: 3.7724800839734083e+18
----------------------------------------
Order = 8


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provid

AIC: 49.3967661872369
BIC: 49.5158517641278
MAPE: 3.7838173445209364e+18
----------------------------------------
Order = 9
AIC: 49.37229000309738
BIC: 49.505825036403984
MAPE: 3.779943648442123e+18
----------------------------------------
Order = 10
AIC: 49.37270071660177
BIC: 49.52068841910852
MAPE: 3.775014652792188e+18
----------------------------------------
Order = 11
AIC: 49.37004318762343
BIC: 49.53248677326129
MAPE: 3.7743253445917005e+18
----------------------------------------
Order = 12
AIC: 49.365882264321606
BIC: 49.54278494816866
MAPE: 3.7747825623992586e+18
----------------------------------------


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provid

In [91]:
print('Best order:', best_order)
print('Best AIC:', best_aic)
print('Best BIC:', best_bic)
print('Overall MAPE:', best_mape)

Best order: 1
Best AIC: 49.365882264321606
Best BIC: 49.48305023814827
Overall MAPE: 3.753278985563126e+18


In [100]:
lag_order = best_model.k_ar
forecast = best_model.forecast(df_train.values[-lag_order:], len(df_test))
forecast_df = pd.DataFrame(forecast, index=df_test.index, columns=cols)

In [106]:
forecast_df

,WEEK,PTNR_RM_CODE,SALES,FLAG
DATE_FIELD,,,,
2019-12-03,43.336956,4.998954e+07,11242.782514,0.942989
2019-12-04,42.699511,4.998656e+07,17677.121907,0.917408
2019-12-05,42.084677,4.998842e+07,21665.406000,0.905794
2019-12-06,41.491709,4.999308e+07,24077.487612,0.900401
2019-12-14,40.920422,4.999929e+07,25496.369699,0.897786
...,...,...,...,...
2023-01-16,27.247063,5.001493e+07,26033.901650,0.887566
2023-01-17,27.247064,5.001492e+07,26034.013232,0.887566
2023-01-18,27.247066,5.001491e+07,26034.124143,0.887566
